# Building Baseline

We are deciding to start with KNN model as it trains quickly, and easily outputs multiple options

In [21]:
import sys
sys.path.append("/user/antonis/code/Ant-mel/legendary_game_recs/")

import pandas as pd
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import MultiLabelBinarizer


In [3]:
final_data = pd.read_json('../raw_data/final_data')


In [7]:
final_data.columns


Index(['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'genres', 'platforms', 'description',
       'total_reviews', 'total_lists', 'main', 'ratings_zero_five',
       'ratings_one_zero', 'ratings_one_five', 'ratings_two_zero',
       'ratings_two_five', 'ratings_three_zero', 'ratings_three_five',
       'ratings_four_zero', 'ratings_four_five', 'ratings_five_zero', 'url',
       'game_id'],
      dtype='object')

In [12]:
# Columns we don't need immediately
cols_to_drop = ['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'platforms', 'description',
       'total_reviews', 'main', 'ratings_zero_five',
       'ratings_one_zero', 'ratings_one_five', 'ratings_two_zero',
       'ratings_two_five', 'ratings_three_zero', 'ratings_three_five',
       'ratings_four_zero', 'ratings_four_five', 'ratings_five_zero', 'url',
       'game_id']


In [11]:
final_useful_columns = final_data.drop(columns=cols_to_drop)
final_useful_columns


,avg_review,genres,total_lists
0,0.0,"[Indie, Racing, Sport]",0
1,0.0,"[Adventure, Indie]",1
2,0.0,[RPG],5
3,0.0,[Adventure],0
4,0.0,[RPG],36
...,...,...,...
37910,3.2,"[Puzzle, Strategy]",10
37911,2.9,"[Puzzle, Tactical]",9
37912,3.5,[Point-and-Click],16
37913,4.0,[Visual Novel],0


In [30]:
# Initilizing encoder and transforming the data
mlb_genre = MultiLabelBinarizer()
transformed_genre = mlb_genre.fit_transform(final_useful_columns['genres'])

# Labeling newly created features
genre_ohe_colums = pd.DataFrame(transformed_genre, columns=mlb_genre.classes_)


In [34]:
# Concatinating the data
final_useful_columns.drop('genres', axis=1, inplace=True)
baseline_data = pd.concat((final_useful_columns, genre_ohe_colums), axis=1)


In [35]:
# Creating baseline
X_train_baseline = baseline_data.drop('avg_review', axis=1)
y_train_baseline = baseline_data['avg_review']

baseline_knn = KNeighborsRegressor(n_neighbors=10).fit(X_train_baseline,
                                                       y_train_baseline)


In [36]:
# Used to get index's to test model
final_data[(final_data['title'] == 'Call of Duty: Black Ops 4')
               | (final_data['title'] == 'The Legend of Zelda: Breath of the Wild')
              | (final_data['title'] == 'Omori')
              | (final_data['title'] == 'FIFA 21')
              | (final_data['title'] == 'Tetris')
              | (final_data['title'] == 'Sid Meier\'s Civilization V')
              | (final_data['title'] == 'Red Dead Redemption')
              | (final_data['title'] == 'Grand Theft Auto V')
                 | (final_data['title'] == 'Pikmin 4')
              | (final_data['title'] == 'The Sims 4')
              | (final_data['title'] == 'Animal Crossing: New Horizons')]


,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,genres,platforms,...,ratings_one_five,ratings_two_zero,ratings_two_five,ratings_three_zero,ratings_three_five,ratings_four_zero,ratings_four_five,ratings_five_zero,url,game_id
841,Red Dead Redemption,1692230400000,40,6,14,35,"[Double Eleven, Rockstar Games]",4.1,"[Adventure, RPG, Shooter]","[PlayStation 4, Nintendo Switch]",...,0,0,0,0,3,2,2,1,/games/red-dead-redemption--2/,260737
906,Pikmin 4,1689897600000,2100,573,1400,2300,[Nintendo],4.3,"[Adventure, Real Time Strategy, Strategy]",[Nintendo Switch],...,5,11,15,52,114,373,520,577,/games/pikmin-4/,59843
2583,Grand Theft Auto V,1655251200000,53,3,8,2,[],4.3,[],"[PlayStation 4, Xbox One, PlayStation 5, Xbox ...",...,0,0,0,1,1,9,9,8,/games/grand-theft-auto-v--2/,239064
5919,FIFA 21,1602115200000,1600,39,57,20,"[Electronic Arts, EA Vancouver]",2.5,"[Simulator, Sport]","[Windows PC, PlayStation 4, Xbox One, PlayStat...",...,90,176,211,255,114,70,18,12,/games/fifa-21/,134101
6988,Animal Crossing: New Horizons,1584576000000,27000,2100,1900,1500,"[Nintendo EPD, Nintendo]",3.6,[Simulator],[Nintendo Switch],...,189,629,1141,2833,3691,4224,1761,1417,/games/animal-crossing-new-horizons/,109462
12552,The Legend of Zelda: Breath of the Wild,1488499200000,44000,3500,6900,3600,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Adventure, Puzzle, RPG]","[Wii U, Nintendo Switch]",...,131,367,563,1257,2163,5236,6908,13704,/games/the-legend-of-zelda-breath-of-the-wild/,7346
16815,The Sims 4,1409616000000,13000,666,1100,139,"[Electronic Arts, The Sims Studio]",3.1,"[RPG, Simulator]","[Windows PC, Mac, PlayStation 4, Xbox One, Pla...",...,202,617,952,1844,1355,938,206,224,/games/the-sims-4/,3212
21323,Sid Meier's Civilization V,1285027200000,7300,70,506,179,"[2K Games, Firaxis Games]",4.0,"[Simulator, Strategy, Turn Based Strategy]","[Windows PC, Mac, Linux]",...,20,50,102,376,683,1485,754,570,/games/sid-meier-s-civilization-v/,866
36255,Omori,1608854400000,11000,1400,6700,5200,"[OMOCAT, PLAYISM]",4.1,"[Adventure, Indie, RPG, Turn Based Strategy]","[Windows PC, Mac, PlayStation 4, Nintendo 3DS,...",...,104,191,254,491,766,1552,1587,3009,/games/omori/,26673
36689,Call of Duty: Black Ops 4,1539302400000,5100,22,414,145,"[Treyarch, Activision]",2.3,[Shooter],"[Windows PC, PlayStation 4, Xbox One]",...,360,598,554,607,296,176,45,23,/games/call-of-duty-black-ops-4/,83727


In [ ]:
baseline_knn.kneighbors()
